# Antenna

This example shows how to create a project in EMIT for
the simulation of an antenna using HFSS.

<img src="_static/emit_simple_cosite.png" width="400">

Keywords: **EMIT**, **Antenna**.

## Perform imports and define constants

Perform required imports.

In [1]:
import tempfile
import time

import ansys.aedt.core

from ansys.aedt.core.emit_core.emit_constants import ResultType, TxRxMode
from ansys.aedt.core.emit_core.nodes.generated import AntennaNode, RadioNode

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Launch AEDT with EMIT

Launch AEDT with EMIT. The ``launch_desktop()`` method initializes AEDT
using the specified version. The second argument can be set to ``True`` to
run AEDT in non-graphical mode.

In [4]:
project_name = ansys.aedt.core.generate_unique_project_name(root_name=temp_folder.name, project_name="antenna_cosite")
d = ansys.aedt.core.launch_desktop(AEDT_VERSION, NG_MODE, new_desktop=True)
aedtapp = ansys.aedt.core.Emit(project_name, version=AEDT_VERSION)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_d927343e-7ecc-41f7-aa49-2702f78cf314.log is enabled.


...

PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerEmit class has been initialized!


PyAEDT INFO: Importing EmitApiPython from: C:\Program Files\ANSYS Inc\v252\AnsysEM\Delcross\EmitApiPython


PyAEDT INFO: Loaded Ansys EMIT API (Beta) 2025.2.0


PyAEDT INFO: Active Design set to EMIT_9W7


## Create and connect EMIT components

Create three radios and connect an antenna to each one.

In [5]:
rad1: RadioNode = aedtapp.schematic.create_component("New Radio")
ant1: AntennaNode = aedtapp.schematic.create_component("Antenna")
if rad1 and ant1:
    aedtapp.schematic.connect_components(rad1.name, ant1.name)

PyAEDT INFO: Using component 'New Radio' from library 'Radios' for type 'New Radio'.


PyAEDT INFO: Using component 'Antenna' from library 'Antennas' for type 'Antenna'.


PyAEDT INFO: Successfully connected components 'Radio' and 'Antenna'.


## Place radio/antenna pair

Use the ``create_radio_antenna()`` method to place the radio/antenna pair. The first
argument is the type of radio. The second argument is the name to
assign to the radio.

In [6]:
rad2, ant2 = aedtapp.schematic.create_radio_antenna("GPS Receiver")
rad3, ant3 = aedtapp.schematic.create_radio_antenna("Bluetooth Low Energy (LE)", "Bluetooth")

PyAEDT INFO: Using component 'GPS Receiver' from library 'Radios\Navigation Systems\GPS-GLONASS' for type 'GPS Receiver'.


PyAEDT INFO: Using component 'Antenna' from library 'Antennas' for type 'Antenna'.


PyAEDT INFO: Successfully connected components 'Antenna 2' and 'GPS Receiver'.


PyAEDT INFO: Using component 'Bluetooth Low Energy (LE)' from library 'Radios\Commercial Unlicensed Systems\Bluetooth' for type 'Bluetooth Low Energy (LE)'.


PyAEDT INFO: Using component 'Antenna' from library 'Antennas' for type 'Antenna'.


PyAEDT INFO: Successfully connected components 'Antenna 3' and 'Bluetooth'.


## Define the RF environment

Specify the RF coupling among antennas.
This functionality is not yet implemented in the API, but it can be entered from the UI.

<img src="_static/coupling.png" width="250">

## Run EMIT simulation

Run the EMIT simulation.

This part of the example requires Ansys AEDT 2023 R2.

In [7]:
# > **Note:** You can uncomment the following code.
#
if AEDT_VERSION > "2023.1":
    rev = aedtapp.results.analyze()
    rx_bands = rev.get_band_names(radio_name=rad2.name, tx_rx_mode=TxRxMode.RX)
    tx_bands = rev.get_band_names(radio_name=rad3.name, tx_rx_mode=TxRxMode.TX)
    domain = aedtapp.results.interaction_domain()
    domain.set_receiver(rad2.name, rx_bands[0], -1)
    domain.set_interferer(rad3.name, tx_bands[0])
    interaction = rev.run(domain)
    worst = interaction.get_worst_instance(ResultType.EMI)
    if worst.has_valid_values():
        emi = worst.get_value(ResultType.EMI)
        print("Worst case interference is: {} dB".format(emi))

PyAEDT INFO: Project antenna_cosite Saved correctly


Worst case interference is: 16.64 dB


## Release AEDT

Release AEDT and close the example.

In [8]:
aedtapp.save_project()
aedtapp.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project antenna_cosite Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``. If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes all temporary files, including the project folder.

In [9]:
temp_folder.cleanup()